# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

__References__


I. [Machine Learning in Action](https://www.manning.com/books/machine-learning-in-action)

II. [Implementing Logistic Regression From Scratch – Part 2: Python Code](https://bryantravissmith.com/2015/12/29/implementing-logistic-regression-from-scratch-part-2-python-code/)

III. [sklearn.linear_model.LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **  

> This is a classification problem, because the outcome variable is discrete (whether the student graduated or didn't graduate) rather than continuous.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
from __future__ import division
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print("Student data read successfully!")

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# Calculate number of students
n_students = student_data.shape[0]

# Calculate number of features.  Subtract 1 to remove the target column
n_features = student_data.shape[1] - 1

# Calculate passing students
n_passed = np.sum(np.where(student_data['passed'] == 'yes', 1, 0))

# Calculate failing students
n_failed = n_students - n_passed

# TODO: Calculate graduation rate
grad_rate = n_passed/n_students

# Print the results
print("Total number of students: {}".format(n_students))
print("Number of features: {}".format(n_features))
print("Number of students who passed: {}".format(n_passed))
print("Number of students who failed: {}".format(n_failed))
print("Graduation rate of the class: {:.2f}%".format(grad_rate * 100))

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
from sklearn.cross_validation import train_test_split

# Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all,
                                                    y_all,
                                                    train_size=num_train/n_students,
                                                    stratify=y_all,
                                                    random_state=99)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])
print("\nChecking target balance...")
print "Training set: {:.2f}%".format(100 * (y_train == 'yes').mean())
print "Testing set: {:.2f}%".format(100 * (y_test == 'yes').mean())

Training set has 300 samples.
Testing set has 95 samples.

Checking target balance...
Training set: 67.00%
Testing set: 67.37%


In [6]:
print "Benchmark:\nF1 score for all 'yes' on students: {:.4f}".format(
    f1_score(y_true = ['yes']*n_passed + ['no']*n_failed, y_pred = ['yes']*n_students, 
             pos_label='yes', average='binary'))

Benchmark:
F1 score for all 'yes' on students: 0.8030


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **

1) Logistic regression
> __General Applications:__
Logistic regression is used widely in many fields, including the medical and social sciences. For example, the Trauma and Injury Severity Score (TRISS), which is widely used to predict mortality in injured patients, was originally developed by Boyd et al. using logistic regression. [Wiki](https://en.wikipedia.org/wiki/Logistic_regression)

> __Strengths:__
Computationally inexpensive, easy to implement, long history of knowledge representation and easy to interpret.

> __Weaknesses:__
Prone to underfitting, may have low accuracy

> __Why should it be applied:__
Generally trusted by a wide audience and has been used in many contexts, including education.  Might be a great option if the underlying functional form of the data is non-linear.

2) Decision Tree
> __General Applications:__  
A decision tree is a flowchart-like structure in which each internal node represents a "test" on an attribute (e.g. whether a coin flip comes up heads or tails), each branch represents the outcome of the test and each leaf node represents a class label (decision taken after computing all attributes). Decision trees are commonly used in operations research, specifically in decision analysis, to help identify a strategy most likely to reach a goal, but are also a popular tool in machine learning. [Wiki](https://en.wikipedia.org/wiki/Decision_tree)


> __Strengths:__
Computationally inexpensive, underlying model is easy to understand learned results, missing values OK, and can deal with irrelevant features

> __Weakness:__
> Prone to over fit

> __Why should it be applied:__
Since the underlying model is easy to understand, given the context of working within an educational setting, the model might be the good because it is easier to get buy-in from stakeholders.

3) K Nearest Neighbor
> __General Applications:__
k-NN is a type of instance-based learning, or lazy learning, where the function is only approximated locally and all computation is deferred until classification. The k-NN algorithm is among the simplest of all machine learning algorithms.  It is commonly used in geo-spatial research.  [Wiki](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)

> __Strengths:__
High accuracy and insensitive to outliers

> __Weaknesses:__
Computationally expensive and requires a lot of memory

> __Why should it be applied:__ Given the context of the data, i.e yearly collected school data, KNN might be a good option because we dont have to re-train the model as often and thus we can bear the computational and memory expense.  

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [7]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))

    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))


### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [8]:
# Import the three supervised learning models from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Initialize the three models
clf_A = LogisticRegression(random_state=99)
clf_B = DecisionTreeClassifier(random_state=99)
clf_C = KNeighborsClassifier()

# Set up the training set sizes
X_train_100 = X_train.iloc[:100]
y_train_100 = y_train.iloc[:100]

X_train_200 = X_train.iloc[:200]
y_train_200 = y_train.iloc[:200]

X_train_300 = X_train.iloc[:300]
y_train_300 = y_train.iloc[:300]

# Execute the 'train_predict' function for each classifier and each training set size
clfs = [clf_A, clf_B, clf_C]
training_sets = [X_train_100, X_train_200, X_train_300]
testing_sets = [y_train_100, y_train_200, y_train_300]

for clf in clfs:
    for x, y in zip(training_sets, testing_sets):
        print(x.shape)
        train_predict(clf, x, y, X_test, y_test)
        print("\n")
    print("-"*90)
    print("\n")
    
print "F1 score for predicting all \"yes\" on test set: {:.4f}".format(f1_score(y_test,
                                                                                ['yes']*len(y_test),
                                                                                pos_label='yes',
                                                                                average='binary'))

(100, 48)
Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0025 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.8154.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7970.


(200, 48)
Training a LogisticRegression using a training set size of 200. . .
Trained model in 0.0021 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.8255.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.8392.


(300, 48)
Training a LogisticRegression using a training set size of 300. . .
Trained model in 0.0025 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.8163.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.8201.


------------------------------------------------------------------------------------------


(100, 48)
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0009 seconds
Made predictions in 0.0004 seconds.
F1 scor

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Logisitic Regression**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 | 0.0003 seconds | 0.0002 seconds | 0.8919 | 0.7402 |
| 200 | 0.0001 seconds | 0.0001 seconds | 0.8464 | 0.7519 |
| 300 | 0.0002 seconds | 0.0002 seconds | 0.8374 | 0.7761 |

** Classifer 2 - Decision Tree**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 | 0.0002 seconds | 0.0002 seconds | 1.0000 | 0.7500 |
| 200 | 0.0002 seconds | 0.0002 seconds | 1.0000 | 0.6885 |
| 300 | 0.0002 seconds | 0.0001 seconds | 1.0000 | 0.6179 |



** Classifer 3 - K Nearest Neighbor**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 | 0.0011 seconds | 0.0009 seconds | 0.8552 | 0.6963 |
| 200 | 0.0034 seconds | 0.0020 seconds | 0.8652 | 0.7407 |
| 300 | 0.0059 seconds | 0.0016 seconds | 0.8764 | 0.7338 |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

> Among the models we used, we feel the best model is Logistic Regression (LR) because of its prediction speed, train-to-test generalizability, and testing scores.  

> Based on the available data,  starting from 100 and moving to 300 samples, LR continuously gets better f1-testing scores. Given the limited resources of the school district, LR's prediction time is faster than K-Nearest Neighbor (KNN) and roughly similar to the Decision Tree (DT).   Which will translate to lower costs because it's less computationally expensive than KNN.  Moreover, the performance of the LR at 300 samples is the highest among the different models at .7761 (which signals that it has done better with more training samples).  And finally, LR's variance from train-to-test scores decreases as we increase the number of training points which speaks to its ability better generalize to the testing data when we add more data points.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*


**Answer: **

> Logistic Regression takes features about past students (eg, their age, gender, etc) and learns to assign a weight to each feature.  We can then plug these "feature weights" into an equation that multiplies the weights with the feature and adds them up.  This final sum is applied to a special function called a sigmoid that outputs probabilities for each student.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [9]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# generate parameters for C
cs = np.linspace(.000000000001, 1.0, 20)

# Create the parameters list you wish to tune
parameters = {'C': cs,
              'penalty': ['l1', 'l2'],
              'solver' : ['liblinear'],
             }

# Initialize the classifier
clf = LogisticRegression(random_state=99)
lr = LogisticRegression(random_state=99)

# Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, param_grid=parameters, scoring=f1_scorer)

# Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
lr.fit(X_train, y_train)

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))
print('\n')
print "Original model has a training F1 score of {:.4f}.".format(predict_labels(lr, X_train, y_train))
print "Original model has a testing F1 score of {:.4f}.".format(predict_labels(lr, X_test, y_test))

/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Made predictions in 0.0002 seconds.
Tuned model has a training F1 score of 0.8207.
Made predictions in 0.0002 seconds.
Tuned model has a testing F1 score of 0.8533.


Made predictions in 0.0002 seconds.
Original model has a training F1 score of 0.8163.
Made predictions in 0.0002 seconds.
Original model has a testing F1 score of 0.8201.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

> The final f1 score for the training is 0.8207 and testing 0.8533.  After optimization we get a way better testing score 0.8533 (tuned) vs 0.8201 (original).  Our results suggest we wont overfit as much as the untuned LR model.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.